### Installing & Importing Libraries

In [2]:
%%capture
!pip install transformers
!pip install einops
!pip install auto-gptq
!pip install sentencepiece
!pip install accelerate
!pip install optimum

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Large Language Models

## 1- TheBloke/Llama-2-13b-Chat-GPTQ

In [12]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


quantize_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

### One-shot Learning

#### Sentence 1

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے کچھ کام باقی ہے مردم شماری ہو چکی ہے حلقہ بندیوں کو انجام دینا ہوگا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے کچھ کام باقی ہے مردم شماری ہو چکی ہے حلقہ بندیوں کو انجام دینا ہوگا"
### Response:
"انہوں

#### Sentence 2

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے محمد رضوان بھی اچھی بیٹنگ کرتا ہے لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=200)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے محمد رضوان بھی اچھی بیٹنگ کرتا ہے لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا"
### R

#### Sentence 3

In [13]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے اس کے بعد کھانا لگ جائے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=180)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے اس کے بعد کھانا لگ جائے گا"
### Response:
"تمام افر

#### Sentence 4

In [14]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=200)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے"
### Res

#### Sentence 5

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میں آج کھانا کھا کر تمہاری طرف آؤں گا لیکن تم میرا انتظار مت کرنا ہوسکتا ہے کہ مجھے دیر ہوجاے آج بہت اہم موضوع پر بات کرنی ہے"
### Response: "میں آج کھانا کھا کر تمہاری طرف آؤں گا۔ لیکن تم میرا انتظار مت کرنا، ہوسکتا ہے کہ مجھے دیر ہوجاے۔ آج بہت اہم موضوع پر بات کرنی ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہے"

#### Sentence 6

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں"
### Response:
"گھر پر ہوں تو مو

#### Sentence 7

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا"
### Response:


#### Sentence 8

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا"
### Response:
"حسن نے تفصیل بتائی

#### Sentence 9

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=200)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری"
### Respon

#### Sentence 10

In [16]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے"
### Response:
"ہم اکثر 

#### Sentence 11

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میرے بچے بھوکے تھے میں تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میرے بچے بھوکے تھے، میں تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی"
### Response:
"میر

#### Sentence 12

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا یہاں کتنی نرم گھاس ہے سونے میں کتنا مزہ آئے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا یہاں کتنی نرم گھاس ہے سونے میں کتنا مزہ آئے گا"
### Response:
"مجھے سخت زمین والا وہ گندا گھر اچھا نہیں ل

#### Sentence 13

In [17]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے لیکن میں اب آپ کو حقیقت بتاتا ہوں جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے لیکن میں اب آپ کو حقیقت بتاتا ہوں جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں"
### Response:
"میری بات

#### Sentence 14

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=200)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا"
### Response:
"گھر بھی سجایا گ

#### Sentence 15

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ایک شخص کے گھر مہمان آئے اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ایک شخص کے گھر مہمان آئے اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے"
### Response:
"ایک شخص کے 

### Generated & Original Sentences

In [18]:
# Generated Sentences

gen_sentence = "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے، کچھ کام باقی ہے مردم شماری ہو چکی ہے، حلقہ بندیوں کو انجام دینا ہوگا۔"

gen_sentence2 = "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے۔ محمد رضوان بھی اچھی بیٹنگ کرتا ہے۔ لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا۔"

gen_sentence3 = "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے۔ اس کے بعد کھانا لگ جائے گا۔"

gen_sentence4 = "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے۔"

gen_sentence5 = "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے۔ مزید میچز جییتنا شرط ہے۔ اس کے بعد ہی فٹبال میں نام بن سکتا ہے۔"

gen_sentence6 = "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں، گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں۔"

gen_sentence7 = "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا، اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا۔"

gen_sentence8 = "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا، جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا۔"

gen_sentence9 = "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا، اور فاختہ کو نشانہ بنایا، فاختہ پتھر لگنے سے شدید زخمی ہو گئی، پھڑپھڑاتی ہوئی زمین پر آ گری۔"

gen_sentence10 = "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے۔"

gen_sentence11 = "میرے بچے بھوکے تھے، میں تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی۔"

gen_sentence12 = "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا یہاں کتنی نرم گھاس ہے، سونے میں کتنا مزہ آئے گا۔"

gen_sentence13 = "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے، لیکن میں اب آپ کو حقیقت بتاتا ہوں، جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں۔"

gen_sentence14 = "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے، اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا۔"

gen_sentence15 = "ایک شخص کے گھر مہمان آئے، اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے۔"

In [19]:
# Original Sentences

orig_sentence =  "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے، کچھ کام باقی ہے، مردم شماری ہو چکی ہے، حلقہ بندیوں کو انجام دینا ہوگا۔"

orig_sentence2 = "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے۔ محمد رضوان بھی اچھی بیٹنگ کرتا ہے، لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا۔"

orig_sentence3 = "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں، ایک راؤنڈ ابھی باقی ہے، اس کے بعد کھانا لگ جائے گا۔"

orig_sentence4 = "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے۔ زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے۔"

orig_sentence5 = "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے۔ مزید میچز جییتنا شرط ہے، اس کے بعد ہی فٹبال میں نام بن سکتا ہے۔"

orig_sentence6 = "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں، گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں۔"

orig_sentence7 = "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا، جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا۔ اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا۔"

orig_sentence8 = "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے۔ پھر یہ سلسلہ اس وقت تک چلتا رہا، جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا۔"

orig_sentence9 = "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا۔ فاختہ پتھر لگنے سے شدید زخمی ہو گئی۔ پھڑپھڑاتی ہوئی زمین پر آ گری۔"

orig_sentence10 = "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے، اور اس میں نقصان ہمارا ہی ہوتا ہے۔"

orig_sentence11 = "میرے بچے بھوکے تھے، میں تو جلدی جلدی جا رہی تھی، اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے، میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی۔"

orig_sentence12 = "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا۔ یہاں کتنی نرم گھاس ہے، سونے میں کتنا مزہ آئے گا۔"

orig_sentence13 = "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے، لیکن میں اب آپ کو حقیقت بتاتا ہوں۔ جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں۔"

orig_sentence14 = "گھر بھی سجایا گیا، خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا۔"

orig_sentence15 = "ایک شخص کے گھر مہمان آئے۔ اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا، تاکہ مہمانوں کے لئے کھیر بنائی جا سکے۔"

### Calculation of Evaluation Metric Scores

In [20]:
import re
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Define a function to clean the text
def clean_text(text):
    # Remove extra spaces and normalize punctuation
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = re.sub(r'،', ',', text)
    text = re.sub(r'۔', '.', text)
    return text

# Define a list of generated sentences and their corresponding original sentences
generated_sentences = [gen_sentence, gen_sentence2, gen_sentence3, gen_sentence4, gen_sentence5,
                       gen_sentence6, gen_sentence7, gen_sentence8, gen_sentence9, gen_sentence10,
                       gen_sentence11, gen_sentence12, gen_sentence13, gen_sentence14, gen_sentence15]

original_sentences = [orig_sentence, orig_sentence2, orig_sentence3, orig_sentence4, orig_sentence5,
                      orig_sentence6, orig_sentence7, orig_sentence8, orig_sentence9, orig_sentence10,
                      orig_sentence11, orig_sentence12, orig_sentence13, orig_sentence14, orig_sentence15]

# Initialize lists to store precision, recall, and F1 scores
precisions = []
recalls = []
f1_scores = []

# Clean and compare each generated sentence with its corresponding original sentence
for generated_sentence, original_sentence in zip(generated_sentences, original_sentences):
    generated_sentence = clean_text(generated_sentence)
    original_sentence = clean_text(original_sentence)

    # Tokenize sentences into characters
    generated_tokens = list(generated_sentence)
    original_tokens = list(original_sentence)

    # Pad or truncate the sequences to have the same length
    max_length = max(len(generated_tokens), len(original_tokens))
    generated_tokens = generated_tokens + [''] * (max_length - len(generated_tokens))
    original_tokens = original_tokens + [''] * (max_length - len(original_tokens))

    # Calculate precision, recall, and F1 score
    precision = precision_score(original_tokens, generated_tokens, average='weighted')
    recall = recall_score(original_tokens, generated_tokens, average='weighted')
    f1 = f1_score(original_tokens, generated_tokens, average='weighted')

    # Append scores to respective lists
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate average scores
average_precision = sum(precisions) / len(precisions)
average_recall = sum(recalls) / len(recalls)
average_f1 = sum(f1_scores) / len(f1_scores)

# Append average scores to the lists
precisions.append(average_precision)
recalls.append(average_recall)
f1_scores.append(average_f1)

# Create a DataFrame to store the scores
data = {'Precision': precisions, 'Recall': recalls, 'F1 Score': f1_scores}
df = pd.DataFrame(data, index=[f'gen_sentence{i}' for i in range(1, 16)] + ['Average Scores'])
df

,Precision,Recall,F1 Score
gen_sentence1,0.504545,0.500000,0.501818
gen_sentence2,0.986559,0.991935,0.988710
gen_sentence3,0.500000,0.500000,0.500000
gen_sentence4,0.388430,0.388430,0.388430
gen_sentence5,0.987179,0.992308,0.989231
gen_sentence6,1.000000,1.000000,1.000000
gen_sentence7,0.253623,0.253623,0.253623
gen_sentence8,0.344538,0.344538,0.344538
gen_sentence9,0.342657,0.342657,0.342657
gen_sentence10,0.716535,0.716535,0.716535


## 2- TheBloke/Llama-2-7b-Chat-GPTQ

In [4]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


quantize_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

### One-shot Learning

#### Sentence 1

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے کچھ کام باقی ہے مردم شماری ہو چکی ہے حلقہ بندیوں کو انجام دینا ہوگا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے کچھ کام باقی ہے مردم شماری ہو چکی ہے حلقہ بندیوں کو انجام دینا ہوگا"
### Response:
"انہ

#### Sentence 2

In [7]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے محمد رضوان بھی اچھی بیٹنگ کرتا ہے لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=150)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے محمد رضوان بھی اچھی بیٹنگ کرتا ہے لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا"
###

#### Sentence 3

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے اس کے بعد کھانا لگ جائے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے اس کے بعد کھانا لگ جائے گا"
### Response:
"تمام ا

#### Sentence 4

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے"
### R

#### Sentence 5

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہ

#### Sentence 6

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں"
### Response:
"گھر پر ہوں تو، م

#### Sentence 7

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا"
### Response:


#### Sentence 8

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا"
### Response:
حسن نے تفصیل بتائی 

#### Sentence 9

In [4]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری"
### Respo

#### Sentence 10

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے"
### Response:
"ہم اکثر خ

#### Sentence 11

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میرے بچے بھوکے تھے میں تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میرے بچے بھوکے تھے میں تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی"
### Response:
"میرے

#### Sentence 12

In [8]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا یہاں کتنی نرم گھاس ہے سونے میں کتنا مزہ آئے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=100)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا یہاں کتنی نرم گھاس ہے سونے میں کتنا مزہ آئے گا"
### Response:
"مجھے سخت زمین والا وہ گندا گھر اچھا نہیں 

#### Sentence 13

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے لیکن میں اب آپ کو حقیقت بتاتا ہوں جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے لیکن میں اب آپ کو حقیقت بتاتا ہوں جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں"
### Response:
"میری بات 

#### Sentence 14

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا"
### Response:
"گھر بھی سجایا گ

#### Sentence 15

In [ ]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ایک شخص کے گھر مہمان آئے اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے"
### Response:
'''

print("\n\n*** Generate:")
​
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=128)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ایک شخص کے گھر مہمان آئے اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے"
### Response:
"ایک شخص کے 

### Generated & Original Sentences

In [1]:
# Generated Sentences

gen_sentence = "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے، کچھ کام باقی ہے، مردم شماری ہو چکی ہے، حلقہ بندیوں کو انجام دینا ہوگا۔"

gen_sentence2 = "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے۔ محمد رضوان بھی اچھی بیٹنگ کرتا ہے، لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا۔"

gen_sentence3 = "تمام افراد سے گزارش ہے۔ کہ وہ اپنی سیٹوں پر بیٹھ جائیں، ایک راؤنڈ ابھی باقی ہے، اس کے بعد کھانا لگ جائے گا۔"

gen_sentence4 = "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے۔ زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے۔"

gen_sentence5 = "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے۔ مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہے۔"

gen_sentence6 = "گھر پر ہوں تو، موبائل دیکھ کر جاگتے ہیں، گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں۔"

gen_sentence7 = "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا، جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا، اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا۔"

gen_sentence8 = "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا"

gen_sentence9 = "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری۔"

gen_sentence10 = "ہم اکثر خود کو عقل کل سمجھتے ہوئے، اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے، اور اس میں نقصان ہمارا ہی ہوتا ہے۔"

gen_sentence11 = "میرے بچے بھوکے تھے، میں تو جلدی جلدی جا رہی تھی، اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے، میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی"

gen_sentence12 = "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا، یہاں کتنی نرم گھاس ہے، سونے میں کتنا مزہ آئے گا۔"

gen_sentence13 = "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے، لیکن میں اب آپ کو حقیقت بتاتا ہوں، جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں۔"

gen_sentence14 = "گھر بھی سجایا گیا، خوب لائٹیں اور غبارے لگائے گئے، اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا۔"

gen_sentence15 = "ایک شخص کے گھر مہمان آئے، اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے۔"

In [10]:
# Original Sentences

orig_sentence =  "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے، کچھ کام باقی ہے، مردم شماری ہو چکی ہے، حلقہ بندیوں کو انجام دینا ہوگا۔"

orig_sentence2 = "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے۔ محمد رضوان بھی اچھی بیٹنگ کرتا ہے، لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا۔"

orig_sentence3 = "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں، ایک راؤنڈ ابھی باقی ہے، اس کے بعد کھانا لگ جائے گا۔"

orig_sentence4 = "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے۔ زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے۔"

orig_sentence5 = "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے۔ مزید میچز جییتنا شرط ہے، اس کے بعد ہی فٹبال میں نام بن سکتا ہے۔"

orig_sentence6 = "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں، گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں۔"

orig_sentence7 = "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا، جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا۔ اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا۔"

orig_sentence8 = "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے۔ پھر یہ سلسلہ اس وقت تک چلتا رہا، جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا۔"

orig_sentence9 = "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا۔ فاختہ پتھر لگنے سے شدید زخمی ہو گئی۔ پھڑپھڑاتی ہوئی زمین پر آ گری۔"

orig_sentence10 = "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے، اور اس میں نقصان ہمارا ہی ہوتا ہے۔"

orig_sentence11 = "میرے بچے بھوکے تھے، میں تو جلدی جلدی جا رہی تھی، اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے، میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی۔"

orig_sentence12 = "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا۔ یہاں کتنی نرم گھاس ہے، سونے میں کتنا مزہ آئے گا۔"

orig_sentence13 = "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے، لیکن میں اب آپ کو حقیقت بتاتا ہوں۔ جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں۔"

orig_sentence14 = "گھر بھی سجایا گیا، خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا۔"

orig_sentence15 = "ایک شخص کے گھر مہمان آئے۔ اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا، تاکہ مہمانوں کے لئے کھیر بنائی جا سکے۔"

### Calculation of Evaluation Metric Scores

In [11]:
import re
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Define a function to clean the text
def clean_text(text):
    # Remove extra spaces and normalize punctuation
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = re.sub(r'،', ',', text)
    text = re.sub(r'۔', '.', text)
    return text

# Define a list of generated sentences and their corresponding original sentences
generated_sentences = [gen_sentence, gen_sentence2, gen_sentence3, gen_sentence4, gen_sentence5,
                       gen_sentence6, gen_sentence7, gen_sentence8, gen_sentence9, gen_sentence10,
                       gen_sentence11, gen_sentence12, gen_sentence13, gen_sentence14, gen_sentence15]

original_sentences = [orig_sentence, orig_sentence2, orig_sentence3, orig_sentence4, orig_sentence5,
                      orig_sentence6, orig_sentence7, orig_sentence8, orig_sentence9, orig_sentence10,
                      orig_sentence11, orig_sentence12, orig_sentence13, orig_sentence14, orig_sentence15]

# Initialize lists to store precision, recall, and F1 scores
precisions = []
recalls = []
f1_scores = []

# Clean and compare each generated sentence with its corresponding original sentence
for generated_sentence, original_sentence in zip(generated_sentences, original_sentences):
    generated_sentence = clean_text(generated_sentence)
    original_sentence = clean_text(original_sentence)

    # Tokenize sentences into characters
    generated_tokens = list(generated_sentence)
    original_tokens = list(original_sentence)

    # Pad or truncate the sequences to have the same length
    max_length = max(len(generated_tokens), len(original_tokens))
    generated_tokens = generated_tokens + [''] * (max_length - len(generated_tokens))
    original_tokens = original_tokens + [''] * (max_length - len(original_tokens))

    # Calculate precision, recall, and F1 score
    precision = precision_score(original_tokens, generated_tokens, average='weighted')
    recall = recall_score(original_tokens, generated_tokens, average='weighted')
    f1 = f1_score(original_tokens, generated_tokens, average='weighted')

    # Append scores to respective lists
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate average scores
average_precision = sum(precisions) / len(precisions)
average_recall = sum(recalls) / len(recalls)
average_f1 = sum(f1_scores) / len(f1_scores)

# Append average scores to the lists
precisions.append(average_precision)
recalls.append(average_recall)
f1_scores.append(average_f1)

# Create a DataFrame to store the scores
data = {'Precision': precisions, 'Recall': recalls, 'F1 Score': f1_scores}
df = pd.DataFrame(data, index=[f'gen_sentence{i}' for i in range(1, 16)] + ['Average Scores'])
df

,Precision,Recall,F1 Score
gen_sentence1,1.000000,1.000000,1.000000
gen_sentence2,1.000000,1.000000,1.000000
gen_sentence3,0.205607,0.205607,0.205607
gen_sentence4,1.000000,1.000000,1.000000
gen_sentence5,0.692308,0.692308,0.692308
gen_sentence6,0.107438,0.107438,0.107438
gen_sentence7,0.996377,0.992754,0.992754
gen_sentence8,0.352941,0.352941,0.352941
gen_sentence9,0.528169,0.528169,0.528169
gen_sentence10,0.257812,0.257812,0.257812


## 3- TheBloke/vicuna-7B-v1.5-GPTQ

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/vicuna-7B-v1.5-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

(…)na-7B-v1.5-GPTQ/resolve/main/config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

(…)GPTQ/resolve/main/generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

(…)-GPTQ/resolve/main/tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)7B-v1.5-GPTQ/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)PTQ/resolve/main/special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

### One-shot learning

#### Sentence 1

In [9]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے کچھ کام باقی ہے مردم شماری ہو چکی ہے حلقہ بندیوں کو انجام دینا ہوگا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے کچھ کام باقی ہے مردم شماری ہو چکی ہے حلقہ بندیوں کو انجام دینا ہوگا"
### Response:
"انہ

#### Sentence 2

In [10]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے محمد رضوان بھی اچھی بیٹنگ کرتا ہے لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے محمد رضوان بھی اچھی بیٹنگ کرتا ہے لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا"
###

#### Sentence 3

In [11]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے اس کے بعد کھانا لگ جائے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے اس کے بعد کھانا لگ جائے گا"
### Response:
> "تمام

#### Sentence 4

In [12]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے"
### R

#### Sentence 5

In [13]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہ

#### Sentence 6

In [14]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں"
### Response:
"گھر پر ہوں تو مو

#### Sentence 7

In [7]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا"
### Response:

#### Sentence 8

In [16]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا"
### Response:
"حسن نے تفصیل بتائی

#### Sentence 9

In [17]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری"
### Respon

#### Sentence 10

In [9]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=200)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے"
### Response:

"ہم اکثر

#### Sentence 11

In [11]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میرے بچے بھوکے تھے میں تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میرے بچے بھوکے تھے میں تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی"
### Response:
"میر

#### Sentence 12

In [12]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا یہاں کتنی نرم گھاس ہے سونے میں کتنا مزہ آئے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا یہاں کتنی نرم گھاس ہے سونے میں کتنا مزہ آئے گا"
### Response:
"مجھے سخت زمین والا وہ گندا گھر، اچھا نہیں

#### Sentence 13

In [21]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے لیکن میں اب آپ کو حقیقت بتاتا ہوں جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے لیکن میں اب آپ کو حقیقت بتاتا ہوں جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں"
### Response:
"میری بات 

#### Sentence 14

In [15]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا"
### Response:
> "گھر بھی سجای

#### Sentence 15

In [23]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ایک شخص کے گھر مہمان آئے اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ایک شخص کے گھر مہمان آئے اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے"
### Response:
"ایک شخص کے 

## Generated & Original Sentences

In [19]:
# Generated Sentences

gen_sentence = "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے، کچھ کام باقی ہے، مردم شماری ہو چکی ہے، حلقہ بندیوں کو انجام دینا ہوگا۔"

gen_sentence2 = "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے۔ محمد رضوان بھی اچھی بیٹنگ کرتا ہے، لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا۔"

gen_sentence3 = "تمام افراد سے گزارش ہے کہ، وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے، اس کے بعد کھانا لگ جائے گا۔"

gen_sentence4 = "کاش، ہمارے ملک میں امن و امان کی فضا قائم ہوجائے۔ زراعت اور معیشت کے مسائل کے علاوہ، اس مسئلے پر بھی توجہ دینے کی ضرورت ہے۔"

gen_sentence5 = "کرکٹ کے ساتھ، ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے۔ مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہے۔"

gen_sentence6 = "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں، گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں۔"

gen_sentence7 = "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا، جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا۔"

gen_sentence8 = "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا۔"

gen_sentence9 = "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری۔"

gen_sentence10 = "ہم اکثر خود کو عقل کل سمجھتے ہوئے، اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے۔"

gen_sentence11 = "میرے بچے بھوکے تھے میں، تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی۔"

gen_sentence12 = "مجھے سخت زمین والا، وہ گندا گھر اچھا نہیں لگتا، یہاں کتنی نرم گھاس ہے، سونے میں کتنا مزہ آئے گا۔"

gen_sentence13 = "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے، لیکن میں اب آپ کو حقیقت بتاتا ہوں جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں۔"

gen_sentence14 = "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا۔"

gen_sentence15 = "ایک شخص کے گھر مہمان آئے، اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے۔"

In [20]:
# Original Sentences

orig_sentence =  "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے، کچھ کام باقی ہے، مردم شماری ہو چکی ہے، حلقہ بندیوں کو انجام دینا ہوگا۔"

orig_sentence2 = "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے۔ محمد رضوان بھی اچھی بیٹنگ کرتا ہے، لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا۔"

orig_sentence3 = "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں، ایک راؤنڈ ابھی باقی ہے، اس کے بعد کھانا لگ جائے گا۔"

orig_sentence4 = "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے۔ زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے۔"

orig_sentence5 = "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے۔ مزید میچز جییتنا شرط ہے، اس کے بعد ہی فٹبال میں نام بن سکتا ہے۔"

orig_sentence6 = "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں، گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں۔"

orig_sentence7 = "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا، جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا۔ اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا۔"

orig_sentence8 = "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے۔ پھر یہ سلسلہ اس وقت تک چلتا رہا، جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا۔"

orig_sentence9 = "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا۔ فاختہ پتھر لگنے سے شدید زخمی ہو گئی۔ پھڑپھڑاتی ہوئی زمین پر آ گری۔"

orig_sentence10 = "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے، اور اس میں نقصان ہمارا ہی ہوتا ہے۔"

orig_sentence11 = "میرے بچے بھوکے تھے، میں تو جلدی جلدی جا رہی تھی، اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے، میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی۔"

orig_sentence12 = "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا۔ یہاں کتنی نرم گھاس ہے، سونے میں کتنا مزہ آئے گا۔"

orig_sentence13 = "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے، لیکن میں اب آپ کو حقیقت بتاتا ہوں۔ جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں۔"

orig_sentence14 = "گھر بھی سجایا گیا، خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا۔"

orig_sentence15 = "ایک شخص کے گھر مہمان آئے۔ اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا، تاکہ مہمانوں کے لئے کھیر بنائی جا سکے۔"

### Calculation of Evaluation Metric Scores

In [21]:
import re
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Define a function to clean the text
def clean_text(text):
    # Remove extra spaces and normalize punctuation
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = re.sub(r'،', ',', text)
    text = re.sub(r'۔', '.', text)
    return text

# Define a list of generated sentences and their corresponding original sentences
generated_sentences = [gen_sentence, gen_sentence2, gen_sentence3, gen_sentence4, gen_sentence5,
                       gen_sentence6, gen_sentence7, gen_sentence8, gen_sentence9, gen_sentence10,
                       gen_sentence11, gen_sentence12, gen_sentence13, gen_sentence14, gen_sentence15]

original_sentences = [orig_sentence, orig_sentence2, orig_sentence3, orig_sentence4, orig_sentence5,
                      orig_sentence6, orig_sentence7, orig_sentence8, orig_sentence9, orig_sentence10,
                      orig_sentence11, orig_sentence12, orig_sentence13, orig_sentence14, orig_sentence15]

# Initialize lists to store precision, recall, and F1 scores
precisions = []
recalls = []
f1_scores = []

# Clean and compare each generated sentence with its corresponding original sentence
for generated_sentence, original_sentence in zip(generated_sentences, original_sentences):
    generated_sentence = clean_text(generated_sentence)
    original_sentence = clean_text(original_sentence)

    # Tokenize sentences into characters
    generated_tokens = list(generated_sentence)
    original_tokens = list(original_sentence)

    # Pad or truncate the sequences to have the same length
    max_length = max(len(generated_tokens), len(original_tokens))
    generated_tokens = generated_tokens + [''] * (max_length - len(generated_tokens))
    original_tokens = original_tokens + [''] * (max_length - len(original_tokens))

    # Calculate precision, recall, and F1 score
    precision = precision_score(original_tokens, generated_tokens, average='weighted')
    recall = recall_score(original_tokens, generated_tokens, average='weighted')
    f1 = f1_score(original_tokens, generated_tokens, average='weighted')

    # Append scores to respective lists
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate average scores
average_precision = sum(precisions) / len(precisions)
average_recall = sum(recalls) / len(recalls)
average_f1 = sum(f1_scores) / len(f1_scores)

# Append average scores to the lists
precisions.append(average_precision)
recalls.append(average_recall)
f1_scores.append(average_f1)

# Create a DataFrame to store the scores
data = {'Precision': precisions, 'Recall': recalls, 'F1 Score': f1_scores}
df = pd.DataFrame(data, index=[f'gen_sentence{i}' for i in range(1, 16)] + ['Average Scores'])
df

,Precision,Recall,F1 Score
gen_sentence1,1.000000,1.000000,1.000000
gen_sentence2,1.000000,1.000000,1.000000
gen_sentence3,0.726415,0.726415,0.726415
gen_sentence4,0.032520,0.032520,0.032520
gen_sentence5,0.400000,0.400000,0.400000
gen_sentence6,1.000000,1.000000,1.000000
gen_sentence7,0.637681,0.637681,0.637681
gen_sentence8,0.352941,0.352941,0.352941
gen_sentence9,0.528169,0.528169,0.528169
gen_sentence10,0.535433,0.535433,0.535433


## 4- TheBloke/vicuna-13B-v1.5-GPTQ

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/vicuna-13B-v1.5-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

(…)a-13B-v1.5-GPTQ/resolve/main/config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

(…)GPTQ/resolve/main/generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

(…)-GPTQ/resolve/main/tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)3B-v1.5-GPTQ/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)PTQ/resolve/main/special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

#### Sentence 1

In [55]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے کچھ کام باقی ہے مردم شماری ہو چکی ہے حلقہ بندیوں کو انجام دینا ہوگا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے کچھ کام باقی ہے مردم شماری ہو چکی ہے حلقہ بندیوں کو انجام دینا ہوگا"
### Response:

"ان

#### Sentence 2

In [56]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے محمد رضوان بھی اچھی بیٹنگ کرتا ہے لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے محمد رضوان بھی اچھی بیٹنگ کرتا ہے لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا"
###

#### Sentence 3

In [59]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے اس کے بعد کھانا لگ جائے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=200)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں ایک راؤنڈ ابھی باقی ہے اس کے بعد کھانا لگ جائے گا"
### Response:
"تمام ا

#### Sentence 4

In [36]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے"
### R

#### Sentence 5

In [38]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے مالی امداد کے طور پر وہ ہمیں کرنا چاہیئے"
### Response: "ہمیں ضرورتمند افراد کی مدد کرنی چاہیئے۔ اپنی استطاعت کے حساب سے جو بھی ہم سے ہوسکتا ہے، مالی امداد کے طور پر، وہ ہمیں کرنا چاہیئے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے مزید میچز جییتنا شرط ہے اس کے بعد ہی فٹبال میں نام بن سکتا ہ

#### Sentence 6

In [39]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں"
### Response:
"گھر پر ہوں تو مو

#### Sentence 7

In [23]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=220)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا"
### Response:

#### Sentence 8

In [41]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا"
### Response:

"حسن نے تفصیل بتائ

#### Sentence 9

In [42]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری"
### Respon

#### Sentence 10

In [27]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے اور اس میں نقصان ہمارا ہی ہوتا ہے"
### Response:
"ہم اکثر 

#### Sentence 11

In [24]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میرے بچے بھوکے تھے میں تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میرے بچے بھوکے تھے میں تو جلدی جلدی جا رہی تھی اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی"
### Response:

"می

#### Sentence 12

In [50]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا یہاں کتنی نرم گھاس ہے سونے میں کتنا مزہ آئے گا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا یہاں کتنی نرم گھاس ہے سونے میں کتنا مزہ آئے گا"
### Response:

"مجھے سخت زمین والا وہ گندا گھر اچھا نہیں 

#### Sentence 13

In [51]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے لیکن میں اب آپ کو حقیقت بتاتا ہوں جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے لیکن میں اب آپ کو حقیقت بتاتا ہوں جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں"
### Response:

"میری بات

#### Sentence 14

In [52]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "گھر بھی سجایا گیا خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا"
### Response:

"گھر بھی سجائیا

#### Sentence 15

In [54]:
prompt_template=f'''### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے، اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ایک شخص کے گھر مہمان آئے اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے"
### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=256)
print(tokenizer.decode(output[0]))



*** Generate:
<s> ### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "دنیا اس کے لئے مسخر کر دی گئی ہے لیکن ظلم و ستم کرنےاپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے"
### Response: "دنیا اس کے لئے مسخر کر دی گئی ہے، لیکن ظلم و ستم کرنے،اپنی حد سے تجاوز کرنے اور فضول خرچی کرنے سے منع بھی کیا گیا ہے۔"

### Instruction: Add appropriate punctuations and sentence boundaries to the following Urdu text in input. Don't include any kind of html tags and don't truncate the output. Provide the actual input that has been asked. "،" "۔" These are the punctuations that need to be added
### Input: "ایک شخص کے گھر مہمان آئے اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے"
### Response:
"ایک شخص کے 

## Generated & Original Sentences

In [28]:
# Generated Sentences

gen_sentence = "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے۔ کچھ کام باقی ہے۔ مردم شماری ہو چکی ہے۔ حلقہ بندیوں کو انجام دینا ہوگا۔"

gen_sentence2 = "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے۔ محمد رضوان بھی اچھی بیٹنگ کرتا ہے، لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا۔"

gen_sentence3 = "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں۔ ایک راؤنڈ ابھی باقی ہے، اس کے بعد کھانا لگ جائے گا۔"

gen_sentence4 = "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے۔ زراعت اور معیشت کے مسائل کے علاوہ، اس مسئلے پر بھی توجہ دینے کی ضرورت ہے۔"

gen_sentence5 = "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے۔ مزید میچز جییتنا شرط ہے، اس کے بعد ہی فٹبال میں نام بن سکتا ہے۔"

gen_sentence6 = "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں، گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں۔"

gen_sentence7 = "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا، جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا، اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا۔"

gen_sentence8 = "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے پھر یہ سلسلہ اس وقت تک چلتا رہا، جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا۔"

gen_sentence9 = "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا، اور فاختہ کو نشانہ بنایا۔ فاختہ پتھر لگنے سے شدید زخمی ہو گئی پھڑپھڑاتی ہوئی زمین پر آ گری۔"

gen_sentence10 = "ہم اکثر خود کو عقل کل سم"

gen_sentence11 = "میرے بچے بھوکے تھے، میں تو جلدی جلدی جا رہی تھی، اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے۔ میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی۔"

gen_sentence12 = "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا، یہاں کتنی نرم گھاس ہے، سونے میں کتنا مزہ آئے گا۔"

gen_sentence13 = "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے، لیکن میں اب آپ کو حقیقت بتاتا ہوں، جب میں گھر سے نکلتا ہوں، تو آنکھیں کھلی رکھتا ہوں۔"

gen_sentence14 = "گھر بھی سجائیا گیا، خوب لائٹیں اور غبارے لگائے گئے، پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا۔"

gen_sentence15 = "ایک شخص کے گھر مہمان آئے، اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا تاکہ مہمانوں کے لئے کھیر بنائی جا سکے۔"

In [29]:
# Original Sentences

orig_sentence =  "انہوں نے کہا کہ اس عمل میں وقت لگتا ہے، کچھ کام باقی ہے، مردم شماری ہو چکی ہے، حلقہ بندیوں کو انجام دینا ہوگا۔"

orig_sentence2 = "بابر اعظم دنیاے کرکٹ کا سب سے اچھا کھلاڑی ہے۔ محمد رضوان بھی اچھی بیٹنگ کرتا ہے، لیکن اسکو نمبر ون ہونے میں ابھی وقت لگے گا۔"

orig_sentence3 = "تمام افراد سے گزارش ہے کہ وہ اپنی سیٹوں پر بیٹھ جائیں، ایک راؤنڈ ابھی باقی ہے، اس کے بعد کھانا لگ جائے گا۔"

orig_sentence4 = "کاش ہمارے ملک میں امن و امان کی فضا قائم ہوجائے۔ زراعت اور معیشت کے مسائل کے علاوہ اس مسئلے پر بھی توجہ دینے کی ضرورت ہے۔"

orig_sentence5 = "کرکٹ کے ساتھ ساتھ پاکستان کی فٹبال ٹیم بھی بہتری کی جانب گامزن ہے۔ مزید میچز جییتنا شرط ہے، اس کے بعد ہی فٹبال میں نام بن سکتا ہے۔"

orig_sentence6 = "گھر پر ہوں تو موبائل دیکھ کر جاگتے ہیں، گھر سے باہر ہوں تو چائے کے ڈھابوں پر چائے پی پی کر دکان دار کی کمائی کرواتے ہیں۔"

orig_sentence7 = "ارتضیٰ بے حد ذہین اور محنتی بچہ تھا، جب کہ احمر قدرے لاپروا اور کھلنڈرے مزاج کا حامل تھا۔ اسے کھیل کود اور شرارت کے سوا کچھ نہ سوجھتا تھا۔"

orig_sentence8 = "حسن نے تفصیل بتائی تو شاہد صاحب مسکرا دیے۔ پھر یہ سلسلہ اس وقت تک چلتا رہا، جب تک حسن پڑھ لکھ کر آرمی میں ملازم نہ ہوا۔"

orig_sentence9 = "شکاری نے اپنی غلیل میں پتھر کا چھوٹا سا ٹکڑا رکھا اور فاختہ کو نشانہ بنایا۔ فاختہ پتھر لگنے سے شدید زخمی ہو گئی۔ پھڑپھڑاتی ہوئی زمین پر آ گری۔"

orig_sentence10 = "ہم اکثر خود کو عقل کل سمجھتے ہوئے اپنے مخلص ساتھیوں اور بزرگوں کی نصیحتوں پر کان نہیں دھرتے، اور اس میں نقصان ہمارا ہی ہوتا ہے۔"

orig_sentence11 = "میرے بچے بھوکے تھے، میں تو جلدی جلدی جا رہی تھی، اس گڑھے پر بہت سارے پتے پڑے ہوئے تھے، میں نے جونہی یہاں چھلانگ ماری تو اس میں گر گئی۔"

orig_sentence12 = "مجھے سخت زمین والا وہ گندا گھر اچھا نہیں لگتا۔ یہاں کتنی نرم گھاس ہے، سونے میں کتنا مزہ آئے گا۔"

orig_sentence13 = "میری بات سن کر آپ ضرور شک میں مبتلا ہوں گے، لیکن میں اب آپ کو حقیقت بتاتا ہوں۔ جب میں گھر سے نکلتا ہوں تو آنکھیں کھلی رکھتا ہوں۔"

orig_sentence14 = "گھر بھی سجایا گیا، خوب لائٹیں اور غبارے لگائے گئے اور پھر چھوٹی زینب نے بڑی زینب کے ساتھ مل کر سالگرہ کا بڑا سا کیک کاٹا۔"

orig_sentence15 = "ایک شخص کے گھر مہمان آئے۔ اس نے ان کی خاطر مدارات کے لئے اپنے ملازم کو دودھ لینے بھیجا، تاکہ مہمانوں کے لئے کھیر بنائی جا سکے۔"

### Calculation of Evaluation Metric Scores

In [30]:
import re
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Define a function to clean the text
def clean_text(text):
    # Remove extra spaces and normalize punctuation
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = re.sub(r'،', ',', text)
    text = re.sub(r'۔', '.', text)
    return text

# Define a list of generated sentences and their corresponding original sentences
generated_sentences = [gen_sentence, gen_sentence2, gen_sentence3, gen_sentence4, gen_sentence5,
                       gen_sentence6, gen_sentence7, gen_sentence8, gen_sentence9, gen_sentence10,
                       gen_sentence11, gen_sentence12, gen_sentence13, gen_sentence14, gen_sentence15]

original_sentences = [orig_sentence, orig_sentence2, orig_sentence3, orig_sentence4, orig_sentence5,
                      orig_sentence6, orig_sentence7, orig_sentence8, orig_sentence9, orig_sentence10,
                      orig_sentence11, orig_sentence12, orig_sentence13, orig_sentence14, orig_sentence15]

# Initialize lists to store precision, recall, and F1 scores
precisions = []
recalls = []
f1_scores = []

# Clean and compare each generated sentence with its corresponding original sentence
for generated_sentence, original_sentence in zip(generated_sentences, original_sentences):
    generated_sentence = clean_text(generated_sentence)
    original_sentence = clean_text(original_sentence)

    # Tokenize sentences into characters
    generated_tokens = list(generated_sentence)
    original_tokens = list(original_sentence)

    # Pad or truncate the sequences to have the same length
    max_length = max(len(generated_tokens), len(original_tokens))
    generated_tokens = generated_tokens + [''] * (max_length - len(generated_tokens))
    original_tokens = original_tokens + [''] * (max_length - len(original_tokens))

    # Calculate precision, recall, and F1 score
    precision = precision_score(original_tokens, generated_tokens, average='weighted')
    recall = recall_score(original_tokens, generated_tokens, average='weighted')
    f1 = f1_score(original_tokens, generated_tokens, average='weighted')

    # Append scores to respective lists
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Calculate average scores
average_precision = sum(precisions) / len(precisions)
average_recall = sum(recalls) / len(recalls)
average_f1 = sum(f1_scores) / len(f1_scores)

# Append average scores to the lists
precisions.append(average_precision)
recalls.append(average_recall)
f1_scores.append(average_f1)

# Create a DataFrame to store the scores
data = {'Precision': precisions, 'Recall': recalls, 'F1 Score': f1_scores}
df = pd.DataFrame(data, index=[f'gen_sentence{i}' for i in range(1, 16)] + ['Average Scores'])
df

,Precision,Recall,F1 Score
gen_sentence1,0.965909,0.972727,0.967273
gen_sentence2,1.000000,1.000000,1.000000
gen_sentence3,0.995283,0.990566,0.990566
gen_sentence4,0.672131,0.672131,0.672131
gen_sentence5,1.000000,1.000000,1.000000
gen_sentence6,1.000000,1.000000,1.000000
gen_sentence7,0.996377,0.992754,0.992754
gen_sentence8,0.344538,0.344538,0.344538
gen_sentence9,0.559859,0.556338,0.557746
gen_sentence10,0.661417,0.188976,0.274513
